<a href="https://colab.research.google.com/github/kuuumaaaa/tabular-playground-series-aug-2021/blob/main/code/EDA01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [52]:
# module
import pandas as pd
import numpy as np
import optuna

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
input = '/content/drive/MyDrive/Colab Notebooks/tabular-playground-series-aug-2021/input/'
output = '/content/drive/MyDrive/Colab Notebooks/tabular-playground-series-aug-2021/output/'

In [42]:
train_ori = pd.read_csv(input+'train.csv')
train = train_ori.copy()
test_ori = pd.read_csv(input+'test.csv')
test = test_ori.copy()

In [54]:
ID = 'id'
TARGET = 'loss'
rm_cols = []
test[TARGET] = np.nan
df = pd.concat([train,test])
# train_x = train.drop('loss', axis=1)
# train_y = train['loss']
# train_y = train_y.reset_index(drop=True)
# test_x = test.copy()
# test_y = test.copy()

In [55]:
import lightgbm as lgb
import optuna.integration.lightgbm as lgbo
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# 特徴量と目的変数をlightgbnのデータ構造に変換する
train_df, val_df = train_test_split( df[df[TARGET].notna()],test_size=0.33, random_state=71)
test_df = df[df[TARGET].isnull()]

feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

train_x = train_df[feat_cols]
train_y = train_df[TARGET]
val_x = val_df[feat_cols]
val_y = val_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

SEED = 0



train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

opt_params = {
    "objective":"regression",
    "metric":"rmse"
}

opt=lgbo.train(
    opt_params,
    train_data,
    valid_sets = val_data,
    verbose_eval=False,
    #ラウンド数
    num_boost_round = 5,
    #打ち切り
    #early_stopping_rounds = 100
)

params = opt.params

model = lgb.train(
    params,
    train_data,
    valid_names=['train', 'valid'],
    valid_sets=[train_data, val_data],
    verbose_eval=100,
)

val_pred = model.predict(val_x, num_iteration=model.best_iteration)
score = mean_absolute_error(val_y, val_pred)

pred_df = pd.DataFrame(sorted(zip(val_x.index, val_pred, val_y)), columns=[
                       'index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(
), train_x.columns)), columns=['importance', 'feature'])

print(f'score: {score:.4f}')

[I 2021-08-24 13:56:36,178] A new study created in memory with name: no-name-929dfaa9-dd8e-45a8-ae31-0fc2ca0d7746
min_data_in_leaf, val_score: 7.913187: 100%|##########| 5/5 [00:24<00:00,  4.92s/it]


[100]	train's rmse: 6.96119	valid's rmse: 7.88658
score: 6.1366


In [ ]:
importance = pd.DataFrame(model.feature_importance(), index=feat_cols, columns=['importance'])
print(importance.sort_values('importance', ascending=False).head(30))